## Step 3 Create Final Network Graph and Assign Link Costs
---
In this notebook, the reconciled network is finalized and link costs are assigned based on link attributes.

## Import modules

In [1]:
from pathlib import Path
import geopandas as gpd
from prepare_network import *
from network_filter import *

## Prepare network by removing isolates and creating a directed network graph

## GDOT Project

In [2]:
project_dir = Path.home() / 'Documents/GitHub/BikewaysimDev/networks'

#bring in network
links = gpd.read_file(project_dir / 'reconciled_network.gpkg',layer='links')
nodes = gpd.read_file(project_dir / 'reconciled_network.gpkg',layer='nodes')

#prepare network 
links, nodes = prepare_network(links,nodes,spd_mph=8)

#if just using time impedance, export as is
nodes.to_file(project_dir/'final_network.gpkg',layer='nodes',driver='GPKG')
links.to_file(project_dir/'final_network.gpkg',layer='links',driver='GPKG')